In [34]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
testcase = 'ur_uint64_join'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/learned_merge_cleanup/sponge/ur_uint64_join/outputs/results/run0', '/home/chesetti/Repos/learned_merge_cleanup/sponge/ur_uint64_join/outputs/results/run1', '/home/chesetti/Repos/learned_merge_cleanup/sponge/ur_uint64_join/outputs/results/run2']


In [35]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_index_build_duration_ns', 'result.inner_index_size',
       'result.outer_disk_fetch', 'result.outer_index_build_duration',
       'result.outer_index_size', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.index.type', 'spec.inner_table',
       'spec.key_size', 'spec.key_type', 'spec.load_sstable_in_mem',
       'spec.name', 'spec.num_threads', 'spec.outer_table', 'spec.result_path',
       'spec.value_size', 'spec.write_result_to_disk'],
      dtype='object')


# Reports

### Overall Duration

In [36]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))

In [37]:
inner_index_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_size', aggfunc='median')
inner_index_size.to_csv(os.path.join(csv_dir, 'inner_index_size.csv'))

In [ ]:
inner_index_build_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_build_duration_ns', aggfunc='median')
inner_index_build_duration.to_csv(os.path.join(csv_dir, 'inner_index_build_duration_ns.csv'))